# Tecnología H3 Uber
Diferentes resoluciones para diferentes aplicaciones

| H3 Resolution | Average Hexagon Area (km<sup>2</sup>) | Average Hexagon Edge Length (km) | Number of unique indexes |
| ------------- | ------------------------------------- | -------------------------------- | ------------------------ |
| 7             | 5.1612932                             | 1.220629759                      | 98,825,162               |
| 8             | 0.7373276                             | 0.461354684                      | 691,776,122              |
| 9             | 0.1053325                             | 0.174375668                      | 4,842,432,842            |
| 10            | 0.0150475                             | 0.065907807                      | 33,897,029,882           |
| 11            | 0.0021496                             | 0.024910561                      | 237,279,209,162          |
| 12            | 0.0003071                             | 0.009415526                      | 1,660,954,464,122        |

In [4]:
import h3

# Write data into Snowflake table
import snowflake.connector

#----------CONNECTION TO SNOWFLAKE-------------
SF_ACCOUNT = 'gfa04036.us-east-1'
SF_WH = 'TRANSFORMING'
SF_USERNAME = 'DBT_USER'
SF_PASSWORD = '2C>`8Q!8y*Sz]h/):Xxy&WNJv'

# Connecting to Snowflake using the default authenticator
ctx = snowflake.connector.connect(
  user=SF_USERNAME,    #username,
  password=SF_PASSWORD,    #password,
  account=SF_ACCOUNT,
  warehouse=SF_WH,
  database='ANALYTICS',
  schema='PROD_STAGING'
)

cur=ctx.cursor()

In [5]:
# Reading data
cur=ctx.cursor()

# Lets aggregate the metering data for Dominion Power up to the day level
#sql = "select date_trunc('day', STARTTIME), count(*) from trips_vw" +\
#        " group by date_trunc('day', STARTTIME)" +\
#        " order by date_trunc('day', STARTTIME)"
sql =   "SELECT \
            uuid, \
            source_country, \
            source_client, \
            latitude AS lat, \
            longitude AS lng \
        FROM ANALYTICS.PROD_MODELED.CLIENTS \
        WHERE latitude IS NOT NULL \
            AND longitude IS NOT NULL \
            AND latitude != 0 \
            AND longitude != 0 "

cur.execute(sql)

# Fetch the result set from the cursor and deliver it as the Pandas DataFrame.
clients = cur.fetch_pandas_all()
clients.reset_index(inplace=True)
clients = clients.to_dict(orient = 'index')

# For each client, try to write the cell
for i in clients:
    clients[i]['r1_h3'] = h3.geo_to_h3(clients[i]['LAT'], clients[i]['LNG'], 1)
    clients[i]['r2_h3'] = h3.geo_to_h3(clients[i]['LAT'], clients[i]['LNG'], 2)
    clients[i]['r3_h3'] = h3.geo_to_h3(clients[i]['LAT'], clients[i]['LNG'], 3)
    clients[i]['r4_h3'] = h3.geo_to_h3(clients[i]['LAT'], clients[i]['LNG'], 4)
    clients[i]['r5_h3'] = h3.geo_to_h3(clients[i]['LAT'], clients[i]['LNG'], 5)
    clients[i]['r6_h3'] = h3.geo_to_h3(clients[i]['LAT'], clients[i]['LNG'], 6)
    clients[i]['r7_h3'] = h3.geo_to_h3(clients[i]['LAT'], clients[i]['LNG'], 7)
    clients[i]['r8_h3'] = h3.geo_to_h3(clients[i]['LAT'], clients[i]['LNG'], 8)
    clients[i]['r9_h3'] = h3.geo_to_h3(clients[i]['LAT'], clients[i]['LNG'], 9)
    clients[i]['r10_h3'] = h3.geo_to_h3(clients[i]['LAT'], clients[i]['LNG'], 10)
    clients[i]['r11_h3'] = h3.geo_to_h3(clients[i]['LAT'], clients[i]['LNG'], 11)
    clients[i]['r12_h3'] = h3.geo_to_h3(clients[i]['LAT'], clients[i]['LNG'], 12)
    clients[i]['r13_h3'] = h3.geo_to_h3(clients[i]['LAT'], clients[i]['LNG'], 13)
    clients[i]['r14_h3'] = h3.geo_to_h3(clients[i]['LAT'], clients[i]['LNG'], 14)
    clients[i]['r15_h3'] = h3.geo_to_h3(clients[i]['LAT'], clients[i]['LNG'], 15)

print(len(clients))

ProgrammingError: 000904 (42000): SQL compilation error: error line 1 at position 92
invalid identifier 'LATITUDE'

In [ ]:
ctx = snowflake.connector.connect(
  user=SF_USERNAME,    #username,
  password=SF_PASSWORD,    #password,
  account=SF_ACCOUNT,
  warehouse=SF_WH,
  database='ANALYTICS',
  schema='PROD_STAGING'
)

cur=ctx.cursor()
sql = "CREATE OR REPLACE TABLE stg_py_clients_h3_cells ( \
                source_country varchar(1020), \
                source_client varchar(1020), \
                client_uuid varchar(1020), \
                r1_h3  varchar(1020), \
                r2_h3  varchar(1020), \
                r3_h3  varchar(1020), \
                r4_h3  varchar(1020), \
                r5_h3  varchar(1020), \
                r6_h3  varchar(1020), \
                r7_h3  varchar(1020), \
                r8_h3  varchar(1020), \
                r9_h3  varchar(1020), \
                r10_h3 varchar(1020), \
                r11_h3 varchar(1020), \
                r12_h3 varchar(1020), \
                r13_h3 varchar(1020), \
                r14_h3 varchar(1020), \
                r15_h3 varchar(1020))" 

cur.execute(sql)
#DataFrame of the needed results to update in the same orde
# multiple records to be updated in tuple format
sql = "INSERT INTO stg_py_clients_h3_cells VALUES"
count = 0
for i in clients:
    count += 1
    sql += "(" \
            + "'" + clients[i]['SOURCE_COUNTRY'] + "'" + "," \
            + "'" + clients[i]['SOURCE_CLIENT'] + "'" + "," \
            + "'" + clients[i]['UUID'] + "'" + "," \
            + "'" + clients[i]['r1_h3'] + "'" + "," \
            + "'" + clients[i]['r2_h3'] + "'" + "," \
            + "'" + clients[i]['r3_h3'] + "'" + "," \
            + "'" + clients[i]['r4_h3'] + "'" + "," \
            + "'" + clients[i]['r5_h3'] + "'" + "," \
            + "'" + clients[i]['r6_h3'] + "'" + "," \
            + "'" + clients[i]['r7_h3'] + "'" + "," \
            + "'" + clients[i]['r8_h3'] + "'" + "," \
            + "'" + clients[i]['r9_h3'] + "'" + "," \
            + "'" + clients[i]['r10_h3'] + "'" + "," \
            + "'" + clients[i]['r11_h3'] + "'" + "," \
            + "'" + clients[i]['r12_h3'] + "'" + "," \
            + "'" + clients[i]['r13_h3'] + "'" + "," \
            + "'" + clients[i]['r14_h3'] + "'" + "," \
            + "'" + clients[i]['r15_h3'] + "'" \
            + ")"
    if (count % 10000 == 0):
                cur.execute(sql)
                print("Flushing 10000 records: ", count)
                sql = "INSERT INTO stg_py_clients_h3_cells VALUES "
    else:
        if count < len(clients):
            sql += ","

cur.execute(sql)    
print("Records updated successfully", count)
cur.close()

Flushing 10000 records:  10000
Flushing 10000 records:  20000
Flushing 10000 records:  30000
Flushing 10000 records:  40000
Flushing 10000 records:  50000
Records updated successfully 50733


True

In [ ]:
import json

distinctCells = set()
resolution = 'r7_h3'
for i in clients:
    distinctCells.add(clients[i][resolution])

# Generación de sub-mapas, uno global
listFeatures = []
for i in distinctCells:
    hexagon = h3.h3_to_geo_boundary(i, geo_json=True)
    listFeatures.append({
            "type": "Feature",
            "properties": {
                "Cell":  i
            },
            "geometry": {
                "type": "Polygon",
                "coordinates": [hexagon]
    }})

with open(resolution+'.geojson', 'w') as fp:
    json.dump({
        "type": "FeatureCollection",
        "name": resolution,
        "crs": { "type": "name", "properties": { "name": resolution+'cells' } },
        "features": listFeatures}
    , fp)